# Goal
Use the Youtube API to pull channel and video data from a popular channel
Visualize the data and run basic analysis

Credit to the tutorial on Youtube API from ClarityCoders youtube: https://www.youtube.com/watch?v=2mSwcRb3KjQ

In [92]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [93]:
#import config file that stores API_KEY (not included in repo)
import config
from googleapiclient.discovery import build
import pandas as pd


In [94]:
# Channel will be SmarterEveryDay
channel_id = "UC6107grRI4m0o2-emgoDnAA"
youtube = build("youtube", "v3", developerKey=config.API_KEY)

## Functions
The approach will be to use the Youtube API in 3 steps:
1. Get the channel stats, and a playlist ID for all uploads on the channel
2. Create a list of video IDs from the uploads playlist
3. Create of list of details for each video


In [95]:
# function to get stats from channel - contains upload playlist ID to use later
def get_channel_stats(youtube, channel_id):
    request = youtube.channels().list(
        part = "snippet,contentDetails,statistics",
        id = channel_id
    )

    response = request.execute()

    return response["items"]

In [96]:
# function to get video data from uploads playlist
def get_video_list(youtube, playlist_id):
    video_list = []

    request = youtube.playlistItems().list(
        part = "snippet,contentDetails",
        playlistId = playlist_id,
        maxResults = 50
    )
    next_page = True

    while next_page:
        response = request.execute()
        data = response["items"]

        for video in data:
            video_id = video["contentDetails"]["videoId"]
            video_list.append(video_id)
        
        if 'nextPageToken' in response.keys():
            next_page = True
            request = youtube.playlistItems().list(
                part = "snippet,contentDetails",
                playlistId = playlist_id,
                maxResults = 50,
                pageToken = response['nextPageToken']
            )
        else:
            next_page = False
    
    return video_list


In [97]:
# Function to get video details
def get_video_details(youtube, video_list):
    video_stats = []
    
    for i in range(0, len(video_list), 50):
        request = youtube.videos().list(
            part = "snippet,contentDetails,statistics",
            id = video_list[i:i+50]
        )

        response = request.execute()

        for video in response["items"]:
            title = video["snippet"]["title"]
            published_date = video["snippet"]["publishedAt"]
            tags = video["snippet"]["tags"]
            tag_count = len(tags)
            view_count = video["statistics"].get('viewCount', 0)
            like_count = video["statistics"].get('likeCount', 0)
            dislike_count = video["statistics"].get('dislikeCount', 0)
            comment_count = video["statistics"].get('commentCount', 0)
            
            video_stats_dict = dict(
                title = title,
                published_date = published_date,
                tags = tags,
                tag_count = tag_count,
                view_count = view_count, 
                like_count = like_count,
                dislike_count = dislike_count, 
                comment_count = comment_count
                )

            video_stats.append(video_stats_dict)

    return video_stats

## Execution

In [98]:
# 1. Get the channel stats and uploads playlist ID
channel_stats = get_channel_stats(youtube, channel_id)
uploads_playlist_id = channel_stats[0]['contentDetails']['relatedPlaylists']['uploads']

# 2. Create a list of video IDs
video_list = get_video_list(youtube, uploads_playlist_id)

# 3. Create a list of video details
video_details = get_video_details(youtube, video_list)

Now that I have our video details in a list of dictionaries, I can create a pandas dataframe to aid in manipulating and visualizing this data. I'll save this dataframe to a csv for future access.

In [100]:
video_df = pd.DataFrame(video_details)

video_df.to_csv('youtube_video_details')